In [1]:
import sys, os
import numpy as np

sys.path.append(os.getcwd())
from config import *
from utils import *
from model import *
os.environ['CUDA_VISIBLE_DEVICES']= '0'

Using TensorFlow backend.
/Users/erattakulangara/virtenvs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/erattakulangara/virtenvs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/erattakulangara/virtenvs/tensorflow/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1

## Provide test Details

In [2]:
test_name = "SampleTest2"
Day = "Monday"
Date = "10_2_2021"
n_epochs="50"
n_images="25"
articulator="Velum" #Airway, Tongue, Velum

model_path = output_file_path(test_name,n_epochs,n_images,Day,Date, articulator)

## Model Architecture

In [3]:
import skimage.io as io
import skimage.transform as trans
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import *
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler



def unet(pretrained_weights=None, input_size=target_input_size):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)
    
    model = Model(inputs=inputs, outputs=conv10)
    
    if(pretrained_weights):
        model.load_weights(pretrained_weights)
        
    return model

## Model Traning

In [4]:
model = unet()
myGenerator = trainGenerator(8,target_Tongue_train  ,'image60' ,'mask60', data_gen_args, target_img_size)
valGenerator = trainGenerator(2,target_Tongue_val,'image', 'mask',data_gen_args, target_img_size  )

model.compile(optimizer = Adam(lr = 1e-4), loss = 'binary_crossentropy', metrics = ['accuracy'])
model_checkpoint = ModelCheckpoint(model_path, monitor='loss',verbose=1, save_best_only=True)
model.fit(myGenerator, steps_per_epoch=50,epochs=40,callbacks=[model_checkpoint])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Found 60 images belonging to 1 classes.
Found 60 images belonging to 1 classes.
Epoch 1/40
 9/50 [====>.........................] - ETA: 21:41 - loss: 517.3571 - acc: 0.3450

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/Users/erattakulangara/virtenvs/tensorflow/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3331, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-4-38b5a2120ada>", line 7, in <module>
    model.fit(myGenerator, steps_per_epoch=50,epochs=40,callbacks=[model_checkpoint])
  File "/Users/erattakulangara/virtenvs/tensorflow/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 673, in fit
    initial_epoch=initial_epoch)
  File "/Users/erattakulangara/virtenvs/tensorflow/lib/python3.6/site-packages/tensorflow/python/keras/engine/training.py", line 1433, in fit_generator
    steps_name='steps_per_epoch')
  File "/Users/erattakulangara/virtenvs/tensorflow/lib/python3.6/site-packages/tensorflow/python/keras/engine/training_generator.py", line 264, in model_iteration
    batch_outs = batch_function(*batch_data)
  File "/Users/erattakulangara/virtenvs/tensorflow/lib/pyth

KeyboardInterrupt: 

In [8]:
import tensorflow as tf
testGene = testGenerator("data/Test/")
trainedModel = tf.keras.models.load_model(model_path, compile=False)
results = trainedModel.predict_generator(testGene,19,verbose=1)
saveResult("Output/Images/Velum",results)

 9/19 [=============>................] - ETA: 3s

InvalidArgumentError:  ValueError: Could not find a format to read the specified file in single-image mode
Traceback (most recent call last):

  File "/Users/erattakulangara/anaconda3/envs/multArtiSeg/lib/python3.6/site-packages/tensorflow/python/ops/script_ops.py", line 249, in __call__
    ret = func(*args)

  File "/Users/erattakulangara/anaconda3/envs/multArtiSeg/lib/python3.6/site-packages/tensorflow/python/autograph/impl/api.py", line 645, in wrapper
    return func(*args, **kwargs)

  File "/Users/erattakulangara/anaconda3/envs/multArtiSeg/lib/python3.6/site-packages/tensorflow/python/data/ops/dataset_ops.py", line 961, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "/Users/erattakulangara/anaconda3/envs/multArtiSeg/lib/python3.6/site-packages/tensorflow/python/keras/engine/data_adapter.py", line 837, in wrapped_generator
    for data in generator_fn():

  File "/Volumes/argon_home/DeepLearningAlgo/U_net_T2_T1_multipleArticulatorSegment